In [2]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 

In [3]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

sar_results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts_SAR/"
sar_results = pd.read_csv(f"{sar_results_dir}/recommended_cities_sar.csv")

In [4]:
results_df.head()

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ..."


In [5]:
sar_results.head()

,model,prompt_id,rec_cities_sar,response_sar
0,llama3point1-instruct,prompt_17_gemini-ui,"['Kaunas', 'Sibiu']",I recommend Kaunas and Sibiu. I recommend the...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,"['Kahramanmaras', 'Nevsehir']","I recommend Baia Mare, Kahramanmaras, and Nev..."
2,llama3point1-instruct,prompt_42_gpt-4o-mini,"['Nalchik', 'Arkhangelsk', 'Amsterdam']","I recommend Nalchik, Arkhangelsk, and Amsterd..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Oradea', 'Sibiu']","I recommend Kaunas, Oradea, and Sibiu. I reco..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,"['Kaunas', 'Rennes', 'Copenhagen']","I recommend Kaunas, Rennes, and Copenhagen fo..."


In [6]:
combined_df = pd.merge(
    left=results_df, 
    right=sar_results,
    how='left',
    left_on=['model', 'prompt_id'],
    right_on=['model', 'prompt_id'],
)

In [7]:
combined_df.head()

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,rec_cities_sar,response_sar
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"['Kaunas', 'Sibiu']",I recommend Kaunas and Sibiu. I recommend the...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"['Kahramanmaras', 'Nevsehir']","I recommend Baia Mare, Kahramanmaras, and Nev..."
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be...","['Nalchik', 'Arkhangelsk', 'Amsterdam']","I recommend Nalchik, Arkhangelsk, and Amsterd..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a...","['Kaunas', 'Oradea', 'Sibiu']","I recommend Kaunas, Oradea, and Sibiu. I reco..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","['Kaunas', 'Rennes', 'Copenhagen']","I recommend Kaunas, Rennes, and Copenhagen fo..."


In [52]:
llama = results_df[results_df['model'] == 'llama3point1-instruct']
mistral = results_df[results_df['model'] == 'mistral-instruct']

### Do both models recommend the same city? In how many cases do they differ?

In [53]:
sort_cities = lambda x: sorted(ast.literal_eval(x))

df = results_df.copy(deep=True)
df['rec_cities'] = df['rec_cities'].apply(sort_cities)
df['rec_cities_sustainable'] = df['rec_cities_sustainable'].apply(sort_cities)

def count_same_response(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = sorted(row['llama3point1-instruct'])
        l2 = sorted(row['mistral-instruct'])

        if len(l1) == len(l2) and l1 == l2:
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values


# count_rec = count_same_response(df, 'rec_cities')
# count_rec

In [54]:
pivot_table = df.pivot_table(index='prompt_id', columns='model', values='rec_cities', aggfunc=lambda x: x)
pivot_table

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,[Innsbruck],[]
prompt_0_gemini-ui,[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Kaunas, Strasbourg, Thessaloniki]",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Sarajevo],[]
prompt_10_gemini-ui,[London],[Antalya]
...,...,...
prompt_8_gemini-ui,[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Bergen],[]
prompt_9_gemini-1.5-pro-001,"[Budapest, Luxembourg, Mykolaiv, Nicosia, Orad...","[Amsterdam, Ankara, Budapest]"


In [55]:
pivot_table_sus = df.pivot_table(index='prompt_id', columns='model', values='rec_cities_sustainable', aggfunc=lambda x: x)
pivot_table_sus

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gemini-ui,[Innsbruck],"[Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Craiova, Gaziantep, Kaunas, Mykolaiv, Oradea,...",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Erzurum],"[Aalborg, Kazan, London, Nalchik]"
prompt_10_gemini-ui,[London],"[Antalya, Mykolaiv]"
...,...,...
prompt_8_gemini-ui,[Milan],"[Barcelona, Braga, Kahramanmaras, Milan, Talli..."
prompt_8_gpt-4o-mini,[Nalchik],"[Bergen, Craiova, Maastricht, Plovdiv, Simfero..."
prompt_9_gemini-1.5-pro-001,"[Budapest, Turku]","[Ankara, Budapest]"


In [56]:
count_non_sus = count_same_response(pivot_table)

In [57]:
count_sus = count_same_response(pivot_table_sus)

In [58]:
count_non_sus

8

In [59]:
count_sus

12

The models rarely agree with their responses. Providing sustainability information does increase the model agreement, albeit marginally, perhaps because of the additional constraint placed on the models to consider the S-Fairness score. WHY??

### How many prompt responses have atleast one city in common between Llama and Mistral?

In [60]:
def count_atleast_one_common_city(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = set(row['llama3point1-instruct'])
        l2 = set(row['mistral-instruct'])

        if len(l1 & l2):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values

In [61]:
one_common = count_atleast_one_common_city(pivot_table)
one_common_sus = count_atleast_one_common_city(pivot_table_sus)

print(one_common, one_common_sus)

98 99


Checking if both models predict atleast one common city fares much better, with both models agreeing approximately 33% of the time

## With SAR

In [9]:
llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']

In [10]:
sort_cities = lambda x: sorted(ast.literal_eval(x))

df = combined_df.copy(deep=True)
df['rec_cities'] = df['rec_cities'].apply(sort_cities)
df['rec_cities_sar'] = df['rec_cities_sar'].apply(sort_cities)

def count_same_response(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = sorted(row['llama3point1-instruct'])
        l2 = sorted(row['mistral-instruct'])

        if len(l1) == len(l2) and l1 == l2:
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values


In [11]:
pivot_table = df.pivot_table(index='prompt_id', columns='model', values='rec_cities', aggfunc=lambda x: x)
pivot_table

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,[Innsbruck],[]
prompt_0_gemini-ui,[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Kaunas, Strasbourg, Thessaloniki]",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Sarajevo],[]
prompt_10_gemini-ui,[London],[Antalya]
...,...,...
prompt_8_gemini-ui,[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Bergen],[]
prompt_9_gemini-1.5-pro-001,"[Budapest, Luxembourg, Mykolaiv, Nicosia, Orad...","[Amsterdam, Ankara, Budapest]"


In [13]:
pivot_table_sar = df.pivot_table(index='prompt_id', columns='model', values='rec_cities_sar', aggfunc=lambda x: x)
pivot_table_sar

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,"[Bergen, Innsbruck, Kars, Kaunas, Nalchik, Plo...","[Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gemini-ui,"[Innsbruck, Plovdiv, Stavanger]","[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Kaunas, Oradea, Sibiu]",[Strasbourg]
prompt_10_gemini-1.5-pro-001,"[Erzurum, Sarajevo, Vitoria-Gasteiz]","[Aalborg, Kazan, London, Nalchik]"
prompt_10_gemini-ui,[Varna],"[Mykolaiv, Simferopol]"
...,...,...
prompt_8_gemini-ui,[Milan],"[Barcelona, Braga, Milan, Tallinn]"
prompt_8_gpt-4o-mini,"[Arkhangelsk, Bergen, Varna]",[]
prompt_9_gemini-1.5-pro-001,[Budapest],"[Amsterdam, Ankara, Vienna]"


In [14]:
count_non_sus = count_same_response(pivot_table)

In [15]:
count_sar = count_same_response(pivot_table_sar)

In [16]:
print(count_non_sus, count_sar)

8 2


In [17]:
def count_atleast_one_common_city(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = set(row['llama3point1-instruct'])
        l2 = set(row['mistral-instruct'])

        if len(l1 & l2):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values

In [18]:
one_common = count_atleast_one_common_city(pivot_table)
one_common_sar = count_atleast_one_common_city(pivot_table_sar)

print(one_common, one_common_sar)

98 121
